# 6 Weighting & Aggregation

> “Weight assignment is one of the most important steps in composite indicator construction.”  
> — _Handbook on Constructing Composite Indicators_

I now have **21 normalised indicators** (after dropping nine that were redundant). I choose **five group‐level shares** that reflect our investment focus:

| Sub-index             | Share | Rationale                                                                                                   |
|-----------------------|:-----:|-------------------------------------------------------------------------------------------------------------|
| **Risk & Volatility** |  30%  | In an investment context, managing downside risk is paramount.  A larger share rewards stability. |
| **Financial Strength**|  20%  | Profitability and balance‐sheet health drive creditworthiness and dividend capacity.                         |
| **Market Performance**|  20%  | Valuation and earnings metrics guide entry/exit timing and relative value across stocks.                     |
| **Growth Potential**  |  15%  | Growth prospects matter but can be more volatile so a moderate weight balances opportunity vs. risk.         |
| **Liquidity & Trading**| 15%  | Liquidity affects execution costs and turnover; it’s important but secondary to risk and profitability.    |

## 6.1 Equal weights by group

The “equal-by-group” approach avoids over-emphasising any one sub-index just because it has more indicators.  I implement it by:

1. Assigning each group its **GROUP_SHARE** (see table above).  
2. Dividing that share equally across the group’s indicators.  

In [21]:
import pandas as pd
import numpy as np
from pathlib import Path

ROOT   = Path("..")
PROC   = ROOT / "data" / "processed"
ART    = ROOT / "artifacts" / "weights"
ART.mkdir(parents=True, exist_ok=True)

df = pd.read_parquet(PROC / "csiai_input_normalized.parquet")
if "ticker" in df.columns:
    df = df.set_index("ticker")

GROUPS = {
    "financial_strength": ["roe","debt_to_equity","current_ratio",
                           "oper_cash_flow","ebitda_margin"],
    "growth_potential":   ["revenue_growth","operating_margin","gross_margin"],
    "market_performance": ["eps","market_cap","price_to_sales","payout_ratio"],
    "risk_volatility":    ["hist_volatility","beta","max_drawdown",
                           "stddev_returns","value_at_risk"],
    "liquidity_trading":  ["avg_volume_30d","bid_ask_spread","volume_growth",
                           "float_shares"],
}

GROUP_SHARE = {
    "financial_strength": 0.20,
    "growth_potential"  : 0.15,
    "market_performance": 0.20,
    "risk_volatility"   : 0.30,
    "liquidity_trading" : 0.15,
}

# equal-by-group weights
w_equal = {}
for group, cols in GROUPS.items():
    share = GROUP_SHARE[group] / len(cols)
    print(f"Group: {group}, Share: {share}")
    w_equal.update({c: share for c in cols})

w_equal = pd.Series(w_equal, name="w_equal_group")
w_equal.to_csv(ART / "weights_equal_group.csv")

Group: financial_strength, Share: 0.04
Group: growth_potential, Share: 0.049999999999999996
Group: market_performance, Share: 0.05
Group: risk_volatility, Share: 0.06
Group: liquidity_trading, Share: 0.0375


### 6.2 PCA-variance weights

For every group I ran a PCA in the multivariate notebook and compute each indicator’s importance as the **variance it contributes** across the retained components:

- **Financial Strength, Growth Potential, Market Performance, Liquidity & Trading**  
  These sub-indices have their variance spread fairly evenly across multiple components (PC1 ≈ 25–35 %, PC2 ≈ 20–30 %, PC3 ≈ 15–30 %, and more).  To capture their full multi-dimensional structure, i weight each variable by  
  \[
    w_q \;=\;\sum_{j=1}^k \lambda_j \, L_{qj}^2
  \]  
  where \(L_{qj}\) is the loading of variable \(q\) on PC \(j\), and \(\lambda_j\) is the variance explained by PC \(j\).  I then normalize so each group’s weights sum to 0.20.

- **Risk & Volatility**  
  Here PC 1 alone explains a dominant share (~63 %) of the total variance; the remaining PCs each contribute < 20 %.  In this case using **only the first component** is both simpler and nearly lossless, so we set  
  \[
    w_q \;=\; L_{q,\text{PC1}}^2
  \]  
  and normalize those squared PC 1 loadings to sum to 0.20.

In [22]:
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import gmean

PCA_DIR = ROOT / "artifacts" / "pca"

def pca_weights(group: str) -> pd.Series:
    L = pd.read_csv(PCA_DIR / f"{group}_loadings.csv", index_col=0)

    if group == "risk_volatility":
        w_raw = L["PC1"].pow(2)
    else:
        evr = (
            pd.read_csv(PCA_DIR / f"{group}_explained_variance.csv",
                        header=None, index_col=0)
              .iloc[:, 0]
        )

        evr.index = [f"PC{i+1}" for i in range(len(evr))]
        evr = evr.reindex(L.columns)

        w_raw = (L.pow(2).mul(evr, axis=1)).sum(axis=1)

    return GROUP_SHARE[group] * w_raw / w_raw.sum()

w_pca = (pd.concat([pca_weights(g) for g in GROUPS], axis=0).rename("w_pca_var"))

w_pca.to_csv(ART / "weights_pca_variance.csv")
print("PCA-variance weights saved: ", ART / "weights_pca_variance.csv")

PCA-variance weights saved:  ../artifacts/weights/weights_pca_variance.csv


### 6 .3 Building sub-index scores 

In [23]:
sub_eq  = {}
sub_pca = {}

for grp, cols in GROUPS.items():
    sub_eq [grp] = df[cols].dot(w_equal[cols])
    sub_pca[grp] = df[cols].dot(w_pca  [cols])

sub_eq_df  = pd.DataFrame(sub_eq ).add_suffix("_equal")
sub_pca_df = pd.DataFrame(sub_pca).add_suffix("_pca")

sub_scores = sub_eq_df.join(sub_pca_df)

sub_ranks = sub_scores.rank(ascending=False).add_suffix("_rank")

SUB_DIR = PROC / "subindex"
SUB_DIR.mkdir(exist_ok=True)

sub_scores.to_parquet(SUB_DIR / "sub_index_scores.parquet")
sub_ranks .to_parquet(SUB_DIR / "sub_index_ranks.parquet")

display(sub_scores.head())
display(sub_ranks.head())

,financial_strength_equal,growth_potential_equal,market_performance_equal,risk_volatility_equal,liquidity_trading_equal,financial_strength_pca,growth_potential_pca,market_performance_pca,risk_volatility_pca,liquidity_trading_pca
ticker,,,,,,,,,,
COOP,0.093887,0.095851,0.099708,0.169011,0.058391,0.095714,0.086311,0.097526,0.167602,0.054018
XYL,0.080699,0.076551,0.131819,0.170104,0.075924,0.078878,0.072450,0.135195,0.168309,0.069022
MAR,0.062319,0.089460,0.132518,0.152278,0.077578,0.054943,0.081727,0.134199,0.143020,0.074124
ALSN,0.098543,0.079301,0.111460,0.165505,0.069640,0.102454,0.074431,0.111889,0.162236,0.071538
DHC,0.101562,0.072002,0.087719,0.147528,0.068206,0.104035,0.069182,0.088920,0.141780,0.069767


,financial_strength_equal_rank,growth_potential_equal_rank,market_performance_equal_rank,risk_volatility_equal_rank,liquidity_trading_equal_rank,financial_strength_pca_rank,growth_potential_pca_rank,market_performance_pca_rank,risk_volatility_pca_rank,liquidity_trading_pca_rank
ticker,,,,,,,,,,
COOP,194.0,23.0,1243.0,492.0,1825.0,215.0,21.0,1290.0,488.0,1969.0
XYL,1590.0,1286.0,419.0,447.0,751.0,1711.0,1289.0,484.0,459.0,1044.0
MAR,2233.0,207.0,398.0,1426.0,628.0,2236.0,205.0,509.0,1492.0,770.0
ALSN,91.0,944.0,960.0,645.0,1162.0,88.0,943.0,975.0,653.0,898.0
DHC,50.0,1926.0,1839.0,1722.0,1250.0,75.0,1927.0,1813.0,1583.0,1004.0


### 6.4 Building the composite score
The final composite score is a weighted sum of the normalised indicators, using the weights from either scheme:
CI_equal = Σ w_equal · x_q
CI_pca   = Σ w_pca   · x_q

- I will use **linear aggregation** to reduce the effect of the weight.

- I will be doing both **linear** and **geometric** aggregation just to see how they compare.

In [ ]:
def linear_index(values, weights):
    return np.dot(values, weights)

def geometric_index(values, weights, eps=1e-9):
    return np.exp(np.sum(weights * np.log(values + eps)))

ci = pd.DataFrame(index=df.index)

ci["CI_equal_lin"] = df.apply(linear_index, axis=1, weights=w_equal)
ci["CI_pca_lin"]   = df.apply(linear_index, axis=1, weights=w_pca)

ci["CI_equal_geo"] = df.apply(geometric_index, axis=1, weights=w_equal)
ci["CI_pca_geo"]   = df.apply(geometric_index, axis=1, weights=w_pca)

ci.to_parquet(PROC / "ci_scores.parquet")

### 6.5 Robustness check

* **Median |Δ-rank|** between the two indices tells me how much a company typically moves if I switch the weight logic.
* A **Spearman ρ matrix** of the four variants (equal vs PCA, linear vs  geometric) reveals whether the overall ordering is stable.

The handbook’s sensitivity suggests these two quick tests as a minimum.  If median |Δ| is small and ρ > 0.9 I can be confident that my index is not held hostage by the exact weighting choice.

In [25]:

rank = ci.rank(ascending=False)
delta = (rank["CI_equal_lin"] - rank["CI_pca_lin"]).abs()
print("Median |Δ-rank|  (Equal vs PCA, linear):",
      delta.median().round(2))

spearman = ci.corr(method="spearman")
display(spearman.round(3))

Median |Δ-rank|  (Equal vs PCA, linear): 108.0


,CI_equal_lin,CI_pca_lin,CI_equal_geo,CI_pca_geo
CI_equal_lin,1.000,0.952,0.800,0.776
CI_pca_lin,0.952,1.000,0.760,0.783
CI_equal_geo,0.800,0.760,1.000,0.982
CI_pca_geo,0.776,0.783,0.982,1.000


### 6.6 Summary

* **Assigned two alternative weight schemes**  
  * *Equal-by-group* – every sub-index gets a certain percentage each of its indicators share that slice equally.  
  * *PCA-variance* – indicator weights ∝ variance explained by their principal components
    (PC 1 only for *Risk & Volatility* as recommended by the scree-test).

* **Built four composite variants**  
  * Linear vs geometric aggregation × Equal vs PCA weights.

* **Performed a first robustness check**  
  * Median absolute rank shift between the two linear variants = **108 places**  - rankings are sensitive to weight selection out of **2241 companies**.  
  * Spearman ρ ≥ 0.76 for every pair of indices  - aggregation form matters, but a bit less than weights.

* **Overall**  
  * I will use the PCA-variance weights with linear aggregation for the final index - because it is variance based, linear and 
    therefore more transparent than the geometric alternative.

In [26]:
SUB_DIR = PROC / "subindex"
sub_eq  = (pd.read_parquet(SUB_DIR / "sub_index_scores.parquet")
             .filter(regex="_pca$")
             .rename(columns=lambda c: c.replace("_pca", "")))

table = (
    sub_eq.assign(CSIAI = ci["CI_pca_lin"])
          .sort_values("CSIAI", ascending=False)
)
table.insert(0, "Rank", range(1, len(table)+1))

FINAL = PROC / "ci_final_table.parquet"
table.to_parquet(FINAL)

print("Final CI table saved to:", FINAL)

display(table)

Final CI table saved to: ../data/processed/ci_final_table.parquet


,Rank,financial_strength,growth_potential,market_performance,risk_volatility,liquidity_trading,CSIAI
ticker,,,,,,,
BRSP,1,0.106293,0.079301,0.149700,0.208302,0.093764,0.637360
AFCG,2,0.092488,0.086156,0.141473,0.218720,0.095227,0.634064
PGRE,3,0.100152,0.076071,0.150910,0.208269,0.094976,0.630377
PAHC,4,0.099874,0.071507,0.137018,0.209351,0.103274,0.621024
DEA,5,0.087929,0.078304,0.151787,0.208250,0.090815,0.617085
...,...,...,...,...,...,...,...
NRIX,2237,0.081019,0.066690,0.091531,0.081759,0.038926,0.359925
AMN,2238,0.084606,0.071254,0.084871,0.081613,0.037562,0.359906
FLYW,2239,0.068326,0.077637,0.090156,0.092584,0.028953,0.357655


## 6.7. References

* Organisation for Economic Co-operation and Development: *Handbook on Constructing Composite Indicators: Methodology and User Guide*, 2008.
* Microsoft Copilot - Note that Copilot is Turned on in this notebook to do some code completion.
* ChatGPT - https://chatgpt.com/share/681d4dc8-36f0-800b-8939-d08f76061782
* **Knowlege Policy** - Amazing guide - https://knowledge4policy.ec.europa.eu/composite-indicators/toolkit_en/navigation-page/10-step-guide_en